In [1]:
!pip install pyspark==3.4.2

In [2]:
!pip install minio

In [3]:
!pip install delta-spark==2.4.0

In [4]:
from pyspark.sql import SparkSession
from delta import *
from minio import Minio
import pyspark.sql.functions as F

In [5]:
spark = (
    SparkSession.builder.master("local[*]").appName("csvToParquet")
    #HADOOP
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    #DELTA
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.jars", "jars/aws-java-sdk-bundle-1.11.1026.jar, \
            jars/hadoop-aws-3.2.0.jar")
    .getOrCreate()
)


24/08/18 23:41:06 WARN Utils: Your hostname, DESKTOP-CV9TLA8 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/18 23:41:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hacs/.cache/pypoetry/virtualenvs/pyspark-delta-gPM1PAZH-py3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hacs/.ivy2/cache
The jars for the packages stored in: /home/hacs/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9ae3c1a2-3051-4fa0-8cc5-a19cef5b4483;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 154ms :: artifacts dl 8ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0

In [6]:
client=Minio("localhost:9000",
             access_key="minioadmin",
             secret_key="minioadmin",
             secure=False
            )


In [7]:
objects=client.list_objects('raw', prefix='CAPES/')

In [8]:
objects=client.list_objects('raw', prefix='CAPES/')
csvs = []
for object in objects:
    csvs.append(object.object_name)

In [9]:
bucket_name = 'raw'

In [10]:
def equalslist(df1, df2):
    return list(set(df1.columns).intersection(set(df2.columns)))

def dfjoin(dataframe1, dataframe2, columnjoin):
    
    equals = equalslist(dataframe1,dataframe2)
    equals.remove(columnjoin)
    
    for column in equals:
        dataframe2 = dataframe2.drop(column)
        
    dataframe_join = dataframe1.join(dataframe2, columnjoin, 'full')
    
    return dataframe_join

In [11]:
count = 0
for csv in csvs:
    print(f'{count} - {csv}')
    count+=1

0 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil.csv
1 - CAPES/2021-a-2024-catalogo-de-teses-e-dissertacoes-brasil.csv
2 - CAPES/2021-a-2024-cursos-da-pos-graduacao-stricto-sensu-no-brasil.csv
3 - CAPES/2021-a-2024-detalhes-da-producao-intelectual-bibliografica-de-programas-de-pos-graduacao.csv
4 - CAPES/2021-a-2024-detalhes-da-producao-intelectual-tecnica-de-programas-de-pos-graduacao-stricto-sensu.csv
5 - CAPES/2021-a-2024-discentes-da-pos-graduacao-stricto-sensu-do-brasil.csv
6 - CAPES/2021-a-2024-docentes-da-pos-graduacao-stricto-sensu-no-brasil.csv
7 - CAPES/2021-a-2024-financiadores-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil.csv
8 - CAPES/2021-a-2024-membros-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil.csv
9 - CAPES/2021-a-2024-producao-intelectual-de-pos-graduacao-stricto-sensu-no-brasil.csv
10 - CAPES/2021-a-2024-programas-da-pos-graduacao-stricto-sensu-no-brasil.csv
11 - CAPE

In [12]:
programas = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[10]}')
programas.count()

24/08/18 23:41:10 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


4710

In [13]:
projetos = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[11]}')
projetos.count()

488718

In [14]:
columnjoin = "CD_PROGRAMA_IES"
join_programa_projetos = dfjoin(projetos, programas, columnjoin)
join_programa_projetos.count()

488730

In [15]:
producoes = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[9]}')

In [16]:
columnjoin = "ID_PROJETO"
join_producoes = dfjoin(producoes, join_programa_projetos, columnjoin)
join_producoes.count()

491180

In [17]:
financiadores = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[7]}')
financiadores.count()

162121

In [18]:
columnjoin = "ID_PROJETO"
join_financiadores = dfjoin(join_producoes, financiadores, columnjoin)
join_financiadores.count()

546725

In [19]:
teses = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[1]}')
teses.count()

82238

In [20]:
columnjoin = "ID_PROJETO"
join_teses = dfjoin(join_financiadores, teses, columnjoin)
join_teses.count()

616038

In [21]:
autores = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[0]}')
autores.count()

4047

In [22]:
columnjoin = "ID_ADD_PRODUCAO_INTELECTUAL"
join_autores = dfjoin(join_teses, autores, columnjoin)
join_autores.count()

617634

In [23]:
membros = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[8]}')
membros.count()

1013387

In [24]:
columnjoin = "ID_PROJETO"
join_membros = dfjoin(join_autores, membros, columnjoin)
join_membros.count()

2884561

In [25]:
discentes = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[5]}')
discentes.count()

420350

In [26]:
discentes = discentes.withColumnRenamed("ID_PESSOA", "ID_PESSOA_DISCENTE")

In [39]:
def dfjoininner(dataframe1, dataframe2, columnjoin):
    
    equals = equalslist(dataframe1,dataframe2)
    equals.remove(columnjoin)
    
    for column in equals:
        dataframe2 = dataframe2.drop(column)
        
    dataframe_join = dataframe1.join(dataframe2, columnjoin, 'inner')
    
    return dataframe_join

In [43]:

dfdiscentes = dfjoininner(join_membros, discentes, "ID_PESSOA_DISCENTE")
dfdiscentes.count()


1752270

In [44]:
bibliografica = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[3]}')
bibliografica.count()

338402

In [49]:
discentes_bibliografica = dfjoininner(dfdiscentes, bibliografica, "ID_ADD_PRODUCAO_INTELECTUAL")
discentes_bibliografica.count()

0

In [47]:
print(1752270+338402)

2090672
